In [1]:
import pandas as pd
import numpy as np
import re
print(pd.__version__)
print(np.__version__)

0.24.2
1.16.2


In [2]:
# 크롤링해 온 국민청원 데이터를 판다스를 통해 읽어온다.
petitions = pd.read_csv('petition.csv',
                        parse_dates=['start', 'end'])
# 데이터의 크기가 어느정도인지 본다.
petitions.shape

(377756, 8)

In [3]:
df=petitions

In [4]:
df.isnull().sum()

article_id    0
start         0
end           0
answered      0
votes         0
category      0
title         0
content       1
dtype: int64

In [5]:
df.shape

(377756, 8)

In [21]:
df.isnull().values.any()

False

In [14]:
df=df.dropna()

In [15]:
df.shape

(377755, 8)

In [18]:
# 전체 데이터 중 투표가 500건 이상인 데이터를 기준으로 가져옵니다. 
# 아웃라이어 데이터 제거를 위해 20만건 이상 데이터도 제거합니다.
petition_remove_outlier = df.loc[(df['votes'] < 1) | 
                                        (df['votes'] > 100)]
petition_remove_outlier.shape

(71416, 8)

In [19]:
df = petition_remove_outlier.copy()

In [20]:
df.describe()

,article_id,answered,votes
count,71416.000000,71416.000000,7.141600e+04
mean,158231.999132,0.000406,7.460335e+02
std,146795.771474,0.020147,1.111777e+04
min,23.000000,0.000000,0.000000e+00
25%,34064.500000,0.000000,0.000000e+00
50%,97239.500000,0.000000,0.000000e+00
75%,271198.250000,0.000000,0.000000e+00
max,492036.000000,1.000000,1.192049e+06


In [22]:
# 기본값을 0으로 세팅, 새로운 컬럼 생성
df['votes_pos_neg'] = 0

In [23]:
# 평균 투표수를 구해 옵니다.
votes_mean = df['votes'].mean()
votes_mean

746.03350789739

In [24]:
# 투표수가 평균을 넘으면 1로 다시 세팅합니다.
df['votes_pos_neg'] = (df['votes'] > 1) == 1

In [25]:
df['votes_pos_neg'].dtypes

dtype('bool')

In [26]:
# 타입을 boolean 에서 int로 변경해 줍니다.
df['votes_pos_neg'] = df['votes_pos_neg'].astype(int)

In [27]:
df[['votes', 'votes_pos_neg']].head()

,votes,votes_pos_neg
2,0,0
4,0,0
7,2137,1
10,0,0
13,679,1


In [28]:
# 샘플로 보고 싶은 인덱스의 번호를 넣어주세요.
sample_index = 7

In [29]:
sample_title = df.loc[sample_index, 'title']
sample_title

'소액주주를 보호해주십시오'

In [30]:
sample_content = petitions.loc[sample_index, 'content']
sample_content

'**  존경하옵는 문재인대통령님께\\n저는 중국원양자원이라는 KOSPI상장사의 소액 주주입니다. 중국원양자원(이하 중원)은 이번 재회계감사에서 의견거절이라는 감사인 판단이 내려져 꿈에도 생각못한 상장폐지를 목전에 두고 있습니다.\\n상장폐지가되면 국내에 상장하여 천문학적인 금액을 취득하여 회사의 규모를 키우고 개인자산을 불린 후 자신의 지분도  다 팔아서 이득을 챙긴 중국인 장화리 사장은 정리매매때 휴지조각의 비용으로 자신의 지분을 다시 사들이고 대주주로서의 지위를 다시 획득하여 온전히 자신의 기업으로 만들수 있습니다.\\n결국 한국 국민들의 피땀어린 투자금으로 중국인 사장의 부를 채워주지만 이에 대해 한국의 금융당국 및 상장과 운영의 주체인 거래소는 누구도 이에대한 책임없이 이를 그냥 덮어버리고만 말것입니다.\\n이로인해 약 2만 5천의 국내소액주주는 피눈물을 흘리며 고통속에 살게 될 것입니다.\\n주식투자는 개인의 판단하에 이루어지는 것이나 투자에 대한 개인의 책임을 묻기전에 그 과정은 규정과 상식에 맞게 공정하고 정의로워야 할 것입니다. 이는 존경하는 문대통령님의 말씀이라 가슴깊이 새기고 있었으나 현재 우리나라의 금융 시스템, 특히 거래소와 회계감사는 그렇지 못한 것 같습니다.\\n이렇게 생각하게된 이유는 이번 회계감사의 주된 의견거절의 이유 때문입니다\\n이번 신한회계법인의 추측적이고 터무니없는 의견거절로 전재산이 한순간에 물거품으로 사라질 위기에 처한 본인을 비롯한 많은 사람들이 죽음이라는 극단적인 선택앞에 내몰려져 억울함과 원통함에 피눈물을 흘리고 있습니다. .\\n기업의 사업성이나 계속 존속여부를 점검하는 감사자인 회계법인은 오직 밝혀진 사실과 그에 따른 증거만을 근거로 공정하게 기업의 존속여부에 대한 의견을 내야하고 유추할 경우에도 당연히 관련 법이나 판례에 근거하여야 합니다.\\n그러나 신한회계법인은 법또는 제도권으로 확정되지 않은 추정사항을 의견거절의\'결정적 이유\'로 명시하였는바 이는 감사자의 \'의견거절\'에 따른 참혹한 결과로 볼때 

In [31]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub(' +', ' ', text)
    return text

In [32]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['수', '현', '있는', '있습니다', '그', '년도', '합니다', '하는', 
             '및', '제', '할', '하고', '더', '대한', '한', '그리고', '월', 
             '저는', '없는', '입니다', '등', '일', '많은', '이런', '것은', 
             '왜','같은', '같습니다', '없습니다', '위해', '한다']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words)

In [33]:
# 샘플데이터에 적용
%time pre_sample_content = preprocessing(sample_content)

Wall time: 3.03 ms


In [34]:
%time df['content_preprocessing'] = df['content'].apply(preprocessing)

Wall time: 5.04 s


In [35]:
%time df['content_preprocessed'] = df['content_preprocessing'].apply(remove_stopwords)

Wall time: 3.46 s


In [36]:
df = df.reset_index()
df.head()

,index,article_id,start,end,answered,votes,category,title,content,votes_pos_neg,content_preprocessing,content_preprocessed
0,2,23,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ...",0,만일 하시는 대통령님 및 각 부처 장관님 주무관님들 안녕하세요 전남 목포에서 자영업...,만일 하시는 대통령님 각 부처 장관님 주무관님들 안녕하세요 전남 목포에서 자영업을 ...
1,4,25,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ...",0,만일 하시는 대통령님 및 각 부처 장관님 주무관님들 안녕하세요 전남 목포에서 자영업...,만일 하시는 대통령님 각 부처 장관님 주무관님들 안녕하세요 전남 목포에서 자영업을 ...
2,7,28,2017-08-19,2017-08-26,0,2137,경제민주화,소액주주를 보호해주십시오,** 존경하옵는 문재인대통령님께\n저는 중국원양자원이라는 KOSPI상장사의 소액 ...,1,존경하옵는 문재인대통령님께 저는 중국원양자원이라는 KOSPI상장사의 소액 주주입니...,존경하옵는 문재인대통령님께 중국원양자원이라는 KOSPI상장사의 소액 주주입니다 중...
3,10,31,2017-08-19,2017-11-17,0,0,보건복지,<담배 제조 및 매매금지 법안>을 제정해 주시길 부탁드립니다.,안녕하세요 대통령님\n금연 관련 정책들이 강화되고 있지만 여전히 흡연자의 수는 줄어...,0,안녕하세요 대통령님 금연 관련 정책들이 강화되고 있지만 여전히 흡연자의 수는 줄어들...,안녕하세요 대통령님 금연 관련 정책들이 강화되고 있지만 여전히 흡연자의 수는 줄어들...
4,13,34,2017-08-19,2017-09-18,0,679,기타,『국가유공자 등 예우 및 지원에 관한법률』상「6.25전몰군경 자녀수당」의 불합리한 ...,(현황)\n우리들 아버지께서는 67여년전 북의 남침으로 조국이 위기에 처했을 때 젊...,1,현황 우리들 아버지께서는 여년전 북의 남침으로 조국이 위기에 처했을 때 젊은 목숨...,현황 우리들 아버지께서는 여년전 북의 남침으로 조국이 위기에 처했을 때 젊은 목숨...


In [37]:
split_count = int(df.shape[0] * 0.7)
split_count

49991

In [38]:
df_train = df[:split_count].copy()
df_train.shape

(49991, 12)

In [39]:
# 학습 세트에서 투표수가 평균보다 많은 건
df_train.loc[df_train['votes_pos_neg'] == 1].shape

(7868, 12)

In [40]:
df_test = df[split_count:].copy()
df_test.shape

(21425, 12)

In [41]:
# 테스트 세트에서 투표수가 평균보다 많은 건
df_test.loc[df_test['votes_pos_neg'] == 1].shape

(9841, 12)

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = None, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋습니다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 2000 # 만들 피처의 수, 단어의 수가 됩니다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=2000, min_df=2,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [43]:
%time 
train_feature_vector = vectorizer.fit_transform(df_train['content_preprocessed'])
train_feature_vector.shape

Wall time: 0 ns


(49991, 2000)

In [44]:
%time 
test_feature_vector = vectorizer.fit_transform(df_test['content_preprocessed'])
test_feature_vector.shape

Wall time: 0 ns


(21425, 2000)

In [45]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

2000


['aa',
 'aid',
 'blog',
 'cctv',
 'cctv를',
 'co',
 'co kr',
 'co kr news',
 'com',
 'com main']

In [46]:
dist = np.sum(train_feature_vector, axis=0)

pd.DataFrame(dist, columns=vocab)

,aa,aid,blog,cctv,cctv를,co,co kr,co kr news,com,com main,...,훨씬,희망을,힘든,힘들게,힘들고,힘들어,힘듭니다,힘없는,힘을,힘이
0,628,576,575,312,1009,966,591,1200,308,321,...,485,820,586,321,308,410,345,367,732,524


In [47]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [48]:
%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

Wall time: 0 ns


(49991, 2000)

In [49]:
%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

Wall time: 0 ns


(21425, 2000)

In [50]:
test_feature_tfidf.shape

(21425, 2000)

In [51]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=42)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [52]:
# 학습에 사용할 y_label 을 넣어준다.
y_label = df_train['votes_pos_neg']
%time forest = forest.fit(train_feature_tfidf, y_label)

Wall time: 22.6 s


In [53]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
score = cross_val_score(forest, train_feature_tfidf, 
                        y_label, cv=k_fold, n_jobs=-1, 
                        scoring=scoring)
score

array([0.85368537, 0.85287057, 0.85617123, 0.85597119, 0.85987197])

In [55]:
round(np.mean(score)*100,2)

85.57

In [56]:
# 테스트 데이터를 넣고 예측한다.
y_pred = forest.predict(test_feature_tfidf)
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [57]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 준다.
output = pd.DataFrame(data={'votes_pos_neg_pred':y_pred})
output.head()

,votes_pos_neg_pred
0,0
1,0
2,0
3,0
4,0


In [58]:
# 0과 1이 어떻게 집계 되었는지 확인한다.
# 실제 데이터에는 답변 대상 건이 있는데 없는 것으로 예측되었다.
output['votes_pos_neg_pred'].value_counts()

0    21206
1      219
Name: votes_pos_neg_pred, dtype: int64

In [59]:
df_test['votes_pos_neg_pred'] = y_pred

In [60]:
df_test['pred_diff'] = np.abs(df_test['votes_pos_neg'] - df_test['votes_pos_neg_pred'])
df_test[['title', 'votes', 'votes_pos_neg', 'votes_pos_neg_pred', 'pred_diff']].head()

,title,votes,votes_pos_neg,votes_pos_neg_pred,pred_diff
49991,보행자 교통사고에 대한 안전시설물,0,0,0,0
49992,어린이집 평가인증 제발 없애주세요.,391,1,0,1
49993,7호선 확정고시 원안대로 조기착공 바랍니다.,887,1,0,1
49994,반대한다! 고양시민은 정수장 앞 골프장 절대 불허!,2042,1,0,1
49995,"대도시 아파트, 학교 관통 지하고속도로사업(민자사업), 과연 안전한가?",309,1,0,1


In [61]:
pred_diff = df_test['pred_diff'].value_counts()
pred_diff

0    11613
1     9812
Name: pred_diff, dtype: int64

In [62]:
print('전체 {}건의 데이터 중 {}건 예측'.format(y_pred.shape[0], pred_diff[0]))

전체 21425건의 데이터 중 11613건 예측


In [63]:
acc = ( pred_diff[0] / y_pred.shape[0] ) * 100 
print('예측 비율 {:.6f}'.format(acc))

예측 비율 54.203034
